# ফিচার ইঞ্জিনিয়ারিং 

এই অধ্যায়ে আমরা মেশিন লার্নিয়ের গুরুত্বপূর্ন অংশ নিয়ে আলোচনা করব তা হচ্ছে ফিচার ইঞ্জিনিয়ারিং।  ফিচার ইঞ্জিনিয়ারং হচ্ছে মেশিন লার্নিং এলগরিদমকে আরো ভালোভাবে কাজ করানোর জন্য ডোমেইন এক্সপার্টিজ ব্যবহার করে নতুন ফিচার তৈরি করার পদ্বতি। ডেটাসেটের দুই বা ততোধিক ফিচার যুক্ত করে নতুন ফিচার তৈরি করা হয় যা মেশিনকে শিখতে সাহায্য করে। এক্ষেত্রে যে ডোমেইনে কাজ করতেছেন সেই ডোমেইনের সম্পর্কে ভালো ধারনা থাকা জরুরী।  আমরা এই অধ্যায়ে কিছু ফিচার ইঞ্জিনিয়ারিং টেকনিক শিখব। 

১) একটি Baseline model তৈরি করা

২) Categorical feature গুলো Encode করা এতে করে মডেল আরো ভালোভাবে শিখতে পারে। 

৩) নতুন ফিচার তৈরি করে মডেলকে আরো ভালোভাবে শেখানো। 

## Baseline model তৈরি করা 

In [11]:
import pandas as pd

In [12]:
dataset = pd.read_csv('ks.csv', parse_dates=['deadline', 'launched'])

In [13]:
dataset.shape

(378661, 15)

In [14]:
dataset.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00


What we can do here is predict if a Kickstarter project will succeed. We get the outcome from the state column. To predict the outcome we can use features such as category, currency, funding goal, country, and when it was launched

## Preparing target column¶
আমাদের ডেটাসেটের state কলামটি লক্ষ্য করি। এই কলামটিকে আমরা টার্গেট কলাম হিসেবে ব্যবহার করব । 

In [15]:
dataset['state'].unique()

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

In [16]:
dataset['state'].value_counts()

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

আমরা এখানে দেখতে পাচ্ছি ডেটাসেটের state কলামটিতে ৬ ধরনের ভ্যারিয়েবল রয়েছে। আমরা এখান থেকে live রোগুলো ফেলে দিবো এবং "successful" কে 1 এবং বাকিগুলোকে 0 করে দিবো । 

Dropping projects that are "live"

Counting "successful" states as outcome = 1

Combining every other state as outcome = 0

In [17]:
# Drop live projects
dataset = dataset.query('state != "live"')

In [18]:
dataset.shape

(375862, 15)

In [19]:
dataset.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [20]:
# Add outcome column, "successful" == 1, others are 0
dataset = dataset.assign(outcome=(dataset['state'] == 'successful').astype(int))

In [21]:
dataset.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00,0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00,0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00,0
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00,0
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00,1
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00,1
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00,0
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00,0
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00,0


In [22]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375862 entries, 0 to 378660
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                375862 non-null  int64         
 1   name              375858 non-null  object        
 2   category          375862 non-null  object        
 3   main_category     375862 non-null  object        
 4   currency          375862 non-null  object        
 5   deadline          375862 non-null  datetime64[ns]
 6   goal              375862 non-null  float64       
 7   launched          375862 non-null  datetime64[ns]
 8   pledged           375862 non-null  float64       
 9   state             375862 non-null  object        
 10  backers           375862 non-null  int64         
 11  country           375862 non-null  object        
 12  usd pledged       372066 non-null  float64       
 13  usd_pledged_real  375862 non-null  float64       
 14  usd_

## Converting timestamps

launched ফিচারটি হচ্ছে দিন সময়ের রেকর্ড। আমি এই ফিচারটিকে ভাগ করে আলাদা আলাদা করে দিন, ঘণ্টা, মাস , বছরের কলাম তৈরি করব এতে মডেলের শিখতে সুবিধা হবে। 

In [70]:
print(type(dataset))
dataset = dataset.assign(hour=dataset.launched.dt.hour,
                         day=dataset.launched.dt.day,
                         month=dataset.launched.dt.month,
                         year=dataset.launched.dt.year)


<class 'pandas.core.frame.DataFrame'>


In [72]:
dataset.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,...,usd_pledged_real,usd_goal_real,outcome,hour,day,month,year,category_labels,currency_labels,country_labels
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,...,0.0,1533.95,0,12,11,8,2015,108,5,9
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,...,2421.0,30000.00,0,4,2,9,2017,93,13,22
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,...,220.0,45000.00,0,0,12,1,2013,93,13,22
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,...,1.0,5000.00,0,3,17,3,2012,90,13,22
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,...,1283.0,19500.00,0,8,4,7,2015,55,13,22


## Preparing categorical variables

মেশিন শুধুমাত্র সংখ্যা থেকে শিখতে পারে। যদি ভ্যারিয়েবল হিসেবে স্ট্রিং থাকে তাহলে তাকে সংখ্যায় রূপান্তরিত করে নিতে হবে। তাই আমরা ক্যাটাগরিক্যাল হ্যান্ডেল করব scikit learn এর LabelEncoder টেকনিক ব্যবহার করে। 


In [25]:
from sklearn.preprocessing import LabelEncoder

cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()

# Apply the label encoder to each column
encoded = dataset[cat_features].apply(encoder.fit_transform)
encoded.head(10)

,category,currency,country
0,108,5,9
1,93,13,22
2,93,13,22
3,90,13,22
4,55,13,22
5,123,13,22
6,58,13,22
7,41,13,22
8,113,13,22
9,39,13,22


In [26]:
# Since dataset and encoded have the same index and I can easily join them
data = dataset[['goal', 'hour', 'day', 'month', 'year', 'outcome']].join(encoded)
data.head()

,goal,hour,day,month,year,outcome,category,currency,country
0,1000.0,12,11,8,2015,0,108,5,9
1,30000.0,4,2,9,2017,0,93,13,22
2,45000.0,0,12,1,2013,0,93,13,22
3,5000.0,3,17,3,2012,0,90,13,22
4,19500.0,8,4,7,2015,0,55,13,22


## Creating training, validation, and test splits

ডেটাসেটিকে ট্রেনিং টেস্ট ডেটা সেটে ভাগ করা । ট্রেনিং পার্ট দিয়ে ডেটাকে শেখাতে হবে , ভ্যালিডেশন পার্ট দিয়ে টেস্ট করা এবং টেস্ট পার্ট দিয়ে চূড়ান্ত টেস্ট করে দেখা মডেলটি কেমন কাজ করছে। 

In [27]:
valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

train = data[:-2 * valid_size]
valid = data[-2 * valid_size:-valid_size]
test = data[-valid_size:]

In [35]:
train.shape, valid.shape, test.shape, valid_size

((300690, 9), (37586, 9), (37586, 9), 37586)

In [61]:
print(train['outcome'].value_counts())
print(valid['outcome'].value_counts())
print(test['outcome'].value_counts())


0    193350
1    107340
Name: outcome, dtype: int64
0    24284
1    13302
Name: outcome, dtype: int64
0    24272
1    13314
Name: outcome, dtype: int64


In [28]:
for each in [train, valid, test]:
    print(f"Outcome fraction = {each.outcome.mean():.4f}")

Outcome fraction = 0.3570
Outcome fraction = 0.3539
Outcome fraction = 0.3542


This looks good, each set is around 35% true outcomes likely because the data was well randomized beforehand. A good way to do this automatically is with sklearn.model_selection.StratifiedShuffleSplit but I don't need to use it here.

## Training a LightGBM model
For this course we'll be using a LightGBM model. This is a tree-based model that typically provides the best performance, even compared to XGBoost. It's also relatively fast to train. We won't do hyperparameter optimization because that isn't the goal of this course. So, our models won't be the absolute best performance you can get. But you'll still see model performance improve as we do feature engineering.

In [29]:
import lightgbm as lgb

feature_cols = train.columns.drop('outcome')

dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10, verbose_eval=False)

In [62]:
feature_cols

Index(['goal', 'hour', 'day', 'month', 'year', 'category', 'currency',
       'country'],
      dtype='object')

In [64]:
dtrain

## Making predictions & evaluating the model
Finally, let's make predictions on the test set with the model and see how well it performs. An important thing to remember is that you can overfit to the validation data. This is why we need a test set that the model never sees until the final evaluation.

In [31]:
from sklearn import metrics
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['outcome'], ypred)


print(f"Test AUC score: {score}")


Test AUC score: 0.747615303004287


# নতুন ফিচার  তৈরি করা 

# Interactions 
নতুন ফিচার তৈরি করার সহজ উপায় হচ্ছে  categorical variable গুলোকে যুক্ত করে দেওয়া। 

In [75]:
interactions = dataset['category'] + "_" + dataset['country']
print(interactions.head(10))


0            Poetry_GB
1    Narrative Film_US
2    Narrative Film_US
3             Music_US
4      Film & Video_US
5       Restaurants_US
6              Food_US
7            Drinks_US
8    Product Design_US
9       Documentary_US
dtype: object


In [77]:
dataset.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,...,usd_pledged_real,usd_goal_real,outcome,hour,day,month,year,category_labels,currency_labels,country_labels
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,...,0.0,1533.95,0,12,11,8,2015,108,5,9
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,...,2421.0,30000.00,0,4,2,9,2017,93,13,22
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,...,220.0,45000.00,0,0,12,1,2013,93,13,22
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,...,1.0,5000.00,0,3,17,3,2012,90,13,22
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,...,1283.0,19500.00,0,8,4,7,2015,55,13,22
